In [15]:
# from lda2vec import LDA2Vec
import tensorflow as tf
from lda2vec.Lda2vec import Lda2vec as mm
tf.reset_default_graph()

model = mm(num_unique_documents = 100, vocab_size = 1000, num_topics = 30, pretrained_embeddings = True)